In [49]:
from jupyter_dash import JupyterDash

In [50]:
import dash
import dash_core_components as dcc
import dash_html_components as html
import pandas as pd
import sqlite3
import ipywidgets as widgets
import re
import json
import urllib
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as stats
import math
from datetime import datetime

In [52]:
JupyterDash.infer_jupyter_proxy_config()

In [53]:
df = pd.read_csv('https://plotly.github.io/datasets/country_indicators.csv')
available_indicators = df['Indicator Name'].unique()

con = sqlite3.connect("/home/alexandru/ManySecured/build/pcap.sqlite")
cur = con.cursor()

In [54]:
def get_packet_mac_arr(mac_address):
    packet_timestamp = []
    packet_length = []
    idx = 0;
    con1 = sqlite3.connect("/home/alexandru/ManySecured/build/pcap.sqlite")
    cur1 = con1.cursor()
    for row in cur1.execute('SELECT timestamp,length FROM eth WHERE ether_dhost="' + mac_address + '" ORDER BY timestamp ASC'):
        packet_timestamp.append(idx)
        packet_length.append(row[1])
        idx = idx + 1
    return (packet_timestamp, packet_length)

def get_packet_ip_arr(ip_address):
    packet_timestamp = []
    packet_length = []
    idx = 0;
    con1 = sqlite3.connect("/home/alexandru/ManySecured/build/pcap.sqlite")
    cur1 = con1.cursor()
    for row in cur1.execute('SELECT timestamp,ip_len FROM ip4 WHERE ip_src="' + ip_address + '" ORDER BY timestamp ASC'):
        packet_timestamp.append(idx)
        packet_length.append(row[1])
        idx = idx + 1
    return (packet_timestamp, packet_length)

# retrieve the distinct mac addresses
mac_dict = {}
for row in cur.execute('SELECT * FROM eth ORDER BY timestamp ASC'):
    mac_dict[row[5]] = 1;
    mac_dict[row[6]] = 1;

mac_list = list(mac_dict.keys())
ip4_dict = {}
for row in cur.execute('SELECT * FROM ip4 ORDER BY timestamp ASC'):
    ip4_dict[row[14]] = 1;
    ip4_dict[row[15]] = 1;

ip4_list = list(ip4_dict.keys())

In [55]:
external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']

app = JupyterDash(__name__, external_stylesheets=external_stylesheets)

# Create server variable with Flask server object for use with gunicorn
server = app.server

app.layout = html.Div([
    html.Div([

        html.Div([
            dcc.Dropdown(
                id='mac-select',
                options=[{'label': i, 'value': i} for i in mac_list],
                value='MAC address'
            )
        ],
        style={'width': '49%', 'display': 'inline-block'}),

        html.Div([
            dcc.Dropdown(
                id='ip-select',
                options=[{'label': i, 'value': i} for i in ip4_list],
                value='IP address'
            )
        ],
        style={'width': '49%', 'float': 'right', 'display': 'inline-block'})
    ], style={
        'borderBottom': 'thin lightgrey solid',
        'backgroundColor': 'rgb(250, 250, 250)',
        'padding': '10px 5px'
    }),
    html.Div([
        dcc.Graph(id='mac-plot'),
        dcc.Graph(id='ip-plot'),
    ], style={'display': 'inline-block', 'width': '49%'}),

])


def create_time_series(x_data, y_data, title):
    return {
        'data': [dict(
            x=x_data,
            y=y_data,
            mode='lines+markers'
        )],
        'layout': {
            'height': 225,
            'margin': {'l': 20, 'b': 30, 'r': 10, 't': 10},
            'annotations': [{
                'x': 0, 'y': 0.85, 'xanchor': 'left', 'yanchor': 'bottom',
                'xref': 'paper', 'yref': 'paper', 'showarrow': False,
                'align': 'left', 'bgcolor': 'rgba(255, 255, 255, 0.5)',
                'text': title
            }],
            'yaxis': {'type': 'linear'},
            'xaxis': {'showgrid': False}
        }
    }


@app.callback(
    dash.dependencies.Output('mac-plot', 'figure'),
    [dash.dependencies.Input('mac-select', 'value')])
def update_y_timeseries(macval):
    (x, y) = get_packet_mac_arr(macval)
    return create_time_series(x, y, "MAC")


@app.callback(
    dash.dependencies.Output('ip-plot', 'figure'),
    [dash.dependencies.Input('ip-select', 'value')])
def update_x_timeseries(ipval):
    (x, y) = get_packet_ip_arr('00:04:00:01:00:06')
    return create_time_series(x, y, "IP")


In [56]:
app.run_server(mode="jupyterlab", host='0.0.0.0', debug=True, port=8050)